In [27]:
import re
from datetime import datetime

In [28]:
with open('r1840_report.txt') as file:
    file_contents = file.read()
    print(file_contents)

       IVS Analysis Report for R1840 ($18MAY02XA) 
       This report is the official IVS analysis report that corresponds to 
       the database maintained by the IVS Data Centers for this session.  
   (Analyzed by David Gordon, Goddard Space Flight Center.
    Spoolfile source: interactive solve/nuSolve analysis.) 


***WARNING:  The correlation statistics 
             could not be included in this report.
             The reason for this failure is:  
                $18MAY02XA NOT FOUND IN DATABASE CATALOG (OR HAS NO ENTRIES) ***


***WARNING:  Some of the observations that were correlated in X-band 
             might not have been correlated in S-band. 
             But the attempt to read the S-band database 
             in order to check this possibility failed. 
             Therefore the counts of the correlated observations 
             and other counts derived from them have not been verified.
             The reason for the failure to read the S-band database is:  
  

In [44]:
with open('r1875-ivs-analysis-report.txt') as file:
    contents = file.read()
    sections = contents.split('-----------------------------------------')

In [45]:
def problemFinder(text_section):
    stations = ['KATH12M', 'YARRA12M', 'HOBART12', 'HOBART26']
    problem_bool = []
    problem_string = []
    for ant in stations:
        regex = ant + '.*\n[\s]{11}.*|' + ant + '.*$'
        problem = re.findall(regex,text_section,re.MULTILINE)
        if len(problem) > 0:
            problem_bool.append(True)
            problem_string.append(problem[0].replace('\n', ""))
        else:
            problem_bool.append(False)
            problem_string.append('')
    return problem_bool, problem_string


In [46]:
problemFinder(sections[0])

([True, True, False, False],
 ['KATH12M had DBBC problems. The ambiguity resolution was extremely           difficult, almost but not quite impossible.',
  'YARRA12M has small clock breaks at ~02:00 and ~11:31 UTC.',
  '',
  ''])

In [47]:
def percent2decimal(percent_string):
    return float(percent_string.strip('%'))/100


def stationPerformance(text_section):
    stations = ['KATH12M', 'YARRA12M', 'HOBART12', 'HOBART26']
    station_performance = []
    for ant in stations:
        regex = ant + ".*"
        performance = re.findall(regex,text_section,re.MULTILINE)
        if len(performance) > 0:
            percentage = [s for s in performance[0].split() if "%" in s]
            performance = percent2decimal(percentage[0])
            station_performance.append(performance)
        else:
            station_performance.append(None)
    
    return station_performance

In [48]:
stationPerformance(sections[2])

[0.645, 0.809, None, None]

In [54]:
def metaData(text_section):
    vgosDBtag = re.findall("(?<=\$).{9}",text_section,re.MULTILINE)
    date = re.findall("(?<=\$).{7}",text_section,re.MULTILINE)
    date = datetime.strptime(date[0], '%y%b%d').strftime('%Y-%m-%d')
    exp_code = re.findall("(?<=Analysis Report for\s)(.*?(?=\s))",text_section,re.MULTILINE)
    analyser = re.findall("\S.*(?=\sAnalysis Report for\s)",text_section,re.MULTILINE)
    return exp_code[0], analyser[0], date, vgosDBtag[0]
    # pretty sure this doesn't work post-2099, but like... you shouldn't be using this trash then... right?

In [55]:
metaData(sections[0])

('R1875', 'IVS', '2019-01-02', '19JAN02XA')

In [56]:
with open('r1875-ivs-analysis-report.txt') as file:
    contents = file.read()
    sections = contents.split('-----------------------------------------')
    
meta = metaData(sections[0])
performance = stationPerformance(sections[2])
problems = problemFinder(sections[0])



In [13]:
import MySQLdb as mariadb
conn = mariadb.connect(user='root', passwd='', db='auscope_test')
cursor = conn.cursor()

OperationalError: (2002, "Can't connect to local MySQL server through socket '/run/mysqld/mysqld.sock' (2)")

In [14]:
sql_command = []
station_id = ['Ke', 'Yg', 'Hb', 'Ho']
for i in range(0, len(performance)):
    if performance[i] == None:
        break
    else:
        sql_station = "INSERT IGNORE INTO " + station_id[i] + " (ExpID, Performance, Date, Problem, Problem_String, Analyser) VALUES ('{}', {},'{}', {}, '{}' , '{}');".format(meta[0], performance[i], meta[2], problems[0][i], problems[1][i], meta[1])
        sql_command.append(sql_station)

# this snippet of code essentially checks if a performance metric exists (and therefore the station participated in the experiment)
# If this is true, it generates an SQL command string for that station. 

In [15]:
sql_command

["INSERT IGNORE INTO Ke (ExpID, Performance, Date, Problem, Problem_String, Analyser) VALUES ('R1875', 0.645,'2019-01-02', True, 'KATH12M had DBBC problems. The ambiguity resolution was extremely           difficult, almost but not quite impossible.' , 'IVS');",
 "INSERT IGNORE INTO Yg (ExpID, Performance, Date, Problem, Problem_String, Analyser) VALUES ('R1875', 0.809,'2019-01-02', True, 'YARRA12M has small clock breaks at ~02:00 and ~11:31 UTC.' , 'IVS');"]

In [16]:
for i in range(0,len(sql_command)):
    cursor.execute(sql_command[i])

conn.commit()
conn.close() 

NameError: name 'cursor' is not defined

In [17]:
conn = mariadb.connect(user='root', passwd='', db='auscope_test')
cursor = conn.cursor()
query = "SELECT ExpID FROM Ke"
cursor.execute(query)
result_list = [item for sublist in cursor.fetchall() for item in sublist]
print(result_list)

OperationalError: (2002, "Can't connect to local MySQL server through socket '/run/mysqld/mysqld.sock' (2)")

In [18]:
station_id = ['Ke', 'Yg', 'Hb', 'Ho']
conn = mariadb.connect(user='root', passwd='')
cursor = conn.cursor()
query = "CREATE OR REPLACE DATABASE auscope_test;"
cursor.execute(query)
conn.commit()
query = "USE auscope_test"
cursor.execute(query)
conn.commit()

for ant in station_id:
    query = "CREATE TABLE IF NOT EXISTS "+ ant + " (ExpID VARCHAR(10) NOT NULL PRIMARY KEY, Performance decimal(4,3) NOT NULL, Date DATETIME, Pos_X decimal(14,2), Pos_Y decimal(14,2), Pos_Z decimal(14,2), Problem BIT(1), Problem_String VARCHAR(100), Analyser VARCHAR(10) NOT NULL);" 
    cursor.execute(query)
    conn.commit()
    
conn.close()
    

OperationalError: (2002, "Can't connect to local MySQL server through socket '/run/mysqld/mysqld.sock' (2)")

In [19]:
station_id = ['Ke', 'Yg', 'Hb', 'Ho']
conn = mariadb.connect(user='root', passwd='')
cursor = conn.cursor()
query = "CREATE DATABASE IF NOT EXISTS auscope_test;"
cursor.execute(query)
conn.commit()
query = "USE auscope_test"
cursor.execute(query)
conn.commit()
for ant in station_id:
    query = "CREATE TABLE IF NOT EXISTS "+ ant + " (ExpID VARCHAR(10) NOT NULL PRIMARY KEY, Performance decimal(4,3) NOT NULL, Date DATETIME, Pos_X decimal(14,2), Pos_Y decimal(14,2), Pos_Z decimal(14,2), Problem BIT(1), Problem_String VARCHAR(100), Analyser VARCHAR(10) NOT NULL);" 
    cursor.execute(query)
    conn.commit()
    
conn.close()

OperationalError: (2002, "Can't connect to local MySQL server through socket '/run/mysqld/mysqld.sock' (2)")